In [1]:
!pip install torch einops numpy transformer_lens==2.11.0 


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [48]:
import torch as t
import torch.nn as nn
import einops
import numpy as np
from transformer_lens import (
    ActivationCache,
    FactoredMatrix,
    HookedTransformer,
    HookedTransformerConfig,
    utils,
)
from transformer_lens.hook_points import HookPoint
import tqdm
import csv
import functools

device = 'cuda'

In [3]:
gpt2_small = HookedTransformer.from_pretrained('gpt2').to(device)
# gpt2_xl = HookedTransformer.from_pretrained('gpt2-XL').to(device)

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


### CAA [Sentences]

In [11]:
sentences = []
with open('sentences.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        sentences.append(tuple(row))

In [27]:
test_cache = gpt2_small.run_with_cache('')[1]
keys = []
for key in test_cache:
    if ('resid_pre' in key) or ('resid_mid' in key):
        keys.append(key)
print(keys)


['blocks.0.hook_resid_pre', 'blocks.0.hook_resid_mid', 'blocks.1.hook_resid_pre', 'blocks.1.hook_resid_mid', 'blocks.2.hook_resid_pre', 'blocks.2.hook_resid_mid', 'blocks.3.hook_resid_pre', 'blocks.3.hook_resid_mid', 'blocks.4.hook_resid_pre', 'blocks.4.hook_resid_mid', 'blocks.5.hook_resid_pre', 'blocks.5.hook_resid_mid', 'blocks.6.hook_resid_pre', 'blocks.6.hook_resid_mid', 'blocks.7.hook_resid_pre', 'blocks.7.hook_resid_mid', 'blocks.8.hook_resid_pre', 'blocks.8.hook_resid_mid', 'blocks.9.hook_resid_pre', 'blocks.9.hook_resid_mid', 'blocks.10.hook_resid_pre', 'blocks.10.hook_resid_mid', 'blocks.11.hook_resid_pre', 'blocks.11.hook_resid_mid']


In [51]:
accum_steering_sentences = t.zeros(len(keys), gpt2_small.cfg.d_model).to(device)

for (english, spanish) in sentences:
    cache_eng = gpt2_small.run_with_cache(english)[1]
    cache_spa = gpt2_small.run_with_cache(spanish)[1]

    for idx, key in enumerate(keys):
        vec_eng = cache_eng[key].squeeze()[-1] #ie. vector at residual-stream-of-last-token
        vec_spa = cache_spa[key].squeeze()[-1]
        steering = vec_spa - vec_eng
        accum_steering_sentences[idx] += steering

In [ ]:
steering_vecs_sentences = accum_steering_sentences / 50

def sentences_hook(res_vector, hook, steering):
    return res_vector + steering

for idx, key in enumerate(keys):
    partial_hook = functools.partial(sentences_hook, steering=steering_vecs_sentences[idx])
    logits = gpt2_small.run_with_hooks(
        'The horse drank some',
        return_type = 'logits',
        fwd_hooks = [(key, partial_hook)],
    )
    out = logits.squeeze().argmax(-1)[-1]
    print(gpt2_small.to_string(out))


 water
 of
 of
 of
 water
 water
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 s
 s
 p
 su
 su
 su
 of


### CAA [Words]

In [59]:
words = []
with open('words.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        words.append(tuple(row))

In [60]:
accum_steering_words = t.zeros_like(accum_steering_sentences)

for (english, spanish) in words:
    cache_eng = gpt2_small.run_with_cache(english)[1]
    cache_spa = gpt2_small.run_with_cache(spanish)[1]

    for idx, key in enumerate(keys):
        vec_eng = cache_eng[key].squeeze()[-1]
        vec_spa = cache_spa[key].squeeze()[-1]
        steering = vec_spa - vec_eng
        accum_steering_words[idx] += steering

In [61]:
steering_vecs_words = accum_steering_words / 50

def words_hook(res_vector, hook, steering):
    return res_vector + steering

for idx, key in enumerate(keys):
    partial_hook = functools.partial(words_hook, steering=steering_vecs_words[idx])
    logits = gpt2_small.run_with_hooks(
        'The horse drank some',
        return_type = 'logits',
        fwd_hooks = [(key, partial_hook)],
    )
    out = logits.squeeze().argmax(-1)[-1]
    print(gpt2_small.to_string(out))


 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 water
 water
 water
 of
 of
 of
 of


### CAA [Paragraph]

### Scratch

In [13]:
def my_hook(res_vector, hook):
    return res_vector + steering

for layer in range(gpt2_small.cfg.n_layers):
    for pos in ['resid_pre', 'resid_mid']:
        vec1 = cache_1[pos, layer].squeeze()[-1]
        vec2 = cache_2[pos, layer].squeeze()[-1]
        steering = vec2 - vec1

        logits = gpt2_small.run_with_hooks(
            'The horse drank some', 
            return_type='logits',
            fwd_hooks=[
                (f'blocks.{layer}.hook_{pos}', my_hook)
            ]
        )

        out = logits.squeeze().argmax(dim=-1)[-1]
        if out != 7050:
            print(gpt2_small.to_string(out))

        


ana
ana
,
,
,
,
 of
 of
 of
,
,
,
 of
,
.
 de
.
.
 de
 de
 de
 de
 de
 de
